## script for combining each cluster assignment and metadata into one file 

In [1]:
import pandas as pd
import dendropy
import numpy as np
import re
import random

In [3]:

newickpath = "../../phylogenetic/old_results/11-07-24/tree.nwk"
clusters = "la_county_clusters.tsv"
metadata = "../cluster_metadata.tsv"

#read in all three files
tree2 = dendropy.Tree.get(file=open(newickpath, "r"), schema="newick")
df = pd.read_csv(clusters, sep ="\t", index_col = 'strain')
meta_df = pd.read_csv(metadata, sep ="\t", index_col = 'accession')

df["location"] = np.nan
df['date'] = np.nan

#for each leaf in the tree, add metadata information into cluster dataset. 
for leaf in tree2.leaf_node_iter():
    taxon_number = re.findall(r"'(.*?)'", str(leaf.taxon), re.DOTALL)
    if taxon_number[0] in list(df.index):
        df.loc[taxon_number[0], 'date'] =  meta_df.loc[taxon_number[0], 'date']
        df.loc[taxon_number[0], 'location'] =  meta_df.loc[taxon_number[0], 'focus_areas']
        df.loc[taxon_number[0], 'lineage'] =  meta_df.loc[taxon_number[0], 'lineage']

        
#removing blanks and NAs for dates
df.replace("", np.nan, inplace=True)
df.dropna(subset = ["date"], inplace=True)


#make sure to change the name for each respective variant build. Will automate soon.
df.to_csv("la_clusters_with_metadata.tsv", sep = "\t")
